In [17]:
import psycopg2
import sqlalchemy
import globalConstants as gConst
import pandas as ps

In [1]:
class SQL_API:
    
    def __init__(self,db,user="postgres",pw="password",port=5432):
    
        self.setupConnection(db,user,pw,port)
        
        
    def setupConnection(self,db,user,pw,port=5432):
        self.connection = psycopg2.connect(
            user=user,
            password=pw,
            host="localhost",
            port=port,database=db
        )
        
    def getMTRecords(self):
        rows = []
        try:
            cur = self.connection.cursor()
            requestString = "SELECT * FROM mt_votes as a INNER JOIN car_images as b ON a.l_img_cars=b.image_id"
            cur.execute(requestString)
            rows = cur.fetchall()
        except Exception as e:
            print(str(e))
        finally:
            cur.close()
        rowDF = ps.DataFrame(rows)
        rowDF.columns = ['assign_id','l_img_safe','r_img_safe','l_img_relax','r_img_relax','l_img_beauty',
                     'r_img_beauty','l_img_greenspace','r_img_greenspace','l_img_cars','r_img_cars',
                     'abs_cont_outcome_safe','abs_cont_outcome_relax','abs_cont_outcome_beauty',
                     'abs_cont_outcome_greenspace','abs_cont_outcome_cars','car_img_id',
                     'car_cat']
        return(rowDF)
    
    def getTSRecords(self,label):
        try:
            cur = self.connection.cursor()
            prefix = "SELECT * FROM mt_tskill_" + label
            postfix = " as a INNER JOIN image_meta as b ON a.image_id = b.image_id "
            requestString = prefix + postfix
            cur.execute(requestString)
            rows = cur.fetchall()
        except Exception as e:
            print(str(e))
        finally:
            cur.close()
        rowDF = ps.DataFrame(rows)
        rowDF.columns = ['image_id','strong_mu_' + label,'strong_sigma_' + label,'mod_mu_' + label,
                         'mod_sigma_' + label, 'slight_mu_' + label, 'slight_sigma_' + label, 'n_' + label,
                         'a','b','c','d','lat','lon','angle','f','urban_cat']
        rowDF = rowDF.drop(columns=['a','b','c','d','f'])
        return(rowDF)
    
    def getImageCat(self):
        try:
            cur = self.connection.cursor()
            requestString = "SELECT image_id,sample_cat,latitude,longitude FROM image_meta"
            cur.execute(requestString)
            rows = cur.fetchall()
        except Exception as e:
            print(str(e))
        finally:
            cur.close()
        rowDF = ps.DataFrame(rows)
        rowDF.columns = ['image_id','urban_cat','lat','lon']
        return(rowDF)